<a href="https://colab.research.google.com/github/rselent/DS-Unit-2-Sprint-1-Linear-Models/blob/master/module2-regression-2/2_1_2_assignment_regression_classification_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]
### ISN'T this the .5% most extreme prices, and .05% lats/longs? o_O

In [0]:
## FEATURE engineering here:

df['total_rooms'] = df['bedrooms'] + df['bathrooms'] + df['dining_room']

df['opulence_level'] = (df['elevator'] + df['hardwood_floors'] + df['doorman']
                         + df['dishwasher'] + df['no_fee'] + df['laundry_in_building']
                         + df['fitness_center'] + df['pre-war'] + df['laundry_in_unit']
                         + df['roof_deck'] + df['outdoor_space'] + df['dining_room']
                         + df['high_speed_internet'] + df['balcony'] + df['swimming_pool']
                         + df['new_construction'] + df['terrace'] + df['exclusive']
                         + df['loft'] + df['garden_patio'] + df['common_outdoor_space'] )

In [14]:
# convert created to datetime format, first
pd.to_datetime( df.created)

# split dataset into train and test subsets
train = df[ df[ 'created'] < '2016-06-01 00:00:00']
test = df[ df[ 'created'] > '2016-05-31 23:59:59']

train.shape, test.shape

((31844, 36), (16973, 36))

In [8]:
###     5 STEPS:     ###

## IMPORT
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

## INSTANTIATE class(es)
linReg = LinearRegression()

## ARRANGE target vector (y) and features (X)
target = 'price'
features = ['bedrooms', 'bathrooms']   # baseline features, most important rooms

yTrain = train[ target]
yTest = test[ target]

xTrain = train[ features]
xTest = test[ features]

## FIT model
linReg.fit( xTrain, yTrain)
yPred = linReg.predict( xTrain)
print( 'Train error: ${0:.2f}'.format( mean_absolute_error( yTrain, yPred)) )

## APPLY model
yPred = linReg.predict( xTest)
print( 'Test error: ${0:.2f}'.format( mean_absolute_error( yTest, yPred)) )

Train error: $818.53
Test error: $825.90


In [15]:
features = ['total_rooms']
xTrain = train[ features]
xTest = test[ features]

linReg.fit( xTrain, yTrain)
yPred = linReg.predict( xTrain)
print( 'Train error: ${0:.2f}'.format( mean_absolute_error( yTrain, yPred)) )

yPred = linReg.predict( xTest)
print( 'Test error: ${0:.2f}'.format( mean_absolute_error( yTest, yPred)) )

Train error: $896.16
Test error: $907.79


In [16]:
features = ['opulence_level']
xTrain = train[ features]
xTest = test[ features]

linReg.fit( xTrain, yTrain)
yPred = linReg.predict( xTrain)
print( 'Train error: ${0:.2f}'.format( mean_absolute_error( yTrain, yPred)) )

yPred = linReg.predict( xTest)
print( 'Test error: ${0:.2f}'.format( mean_absolute_error( yTest, yPred)) )

Train error: $1141.23
Test error: $1129.40


In [17]:
features = ['total_rooms', 'opulence_level']
xTrain = train[ features]
xTest = test[ features]

linReg.fit( xTrain, yTrain)
yPred = linReg.predict( xTrain)
print( 'Train error: ${0:.2f}'.format( mean_absolute_error( yTrain, yPred)) )

yPred = linReg.predict( xTest)
print( 'Test error: ${0:.2f}'.format( mean_absolute_error( yTest, yPred)) )

Train error: $870.97
Test error: $879.94


In [18]:
features = ['elevator', 'hardwood_floors', 'doorman', 'dishwasher', 'no_fee',
            'laundry_in_building', 'fitness_center', 'pre-war', 'laundry_in_unit',
            'roof_deck', 'outdoor_space', 'dining_room', 'high_speed_internet',
            'balcony', 'swimming_pool', 'new_construction', 'terrace', 
            'exclusive', 'loft', 'garden_patio', 'common_outdoor_space']

xTrain = train[ features]
xTest = test[ features]

linReg.fit( xTrain, yTrain)
yPred = linReg.predict( xTrain)
print( 'Train error: ${0:.2f}'.format( mean_absolute_error( yTrain, yPred)) )

yPred = linReg.predict( xTest)
print( 'Test error: ${0:.2f}'.format( mean_absolute_error( yTest, yPred)) )

## INTERESTING how this results in a different mae than 
## the combined/engineered feature 'opulence_level'...

Train error: $1087.30
Test error: $1082.30


In [26]:
features = ['cats_allowed', 'dogs_allowed', 'opulence_level', 'bedrooms', 'bathrooms']

xTrain = train[ features]
xTest = test[ features]

linReg.fit( xTrain, yTrain)
yPred = linReg.predict( xTrain)
print( 'Train error: ${0:.2f}'.format( mean_absolute_error( yTrain, yPred)) )

yPred = linReg.predict( xTest)
print( 'Test error: ${0:.2f}'.format( mean_absolute_error( yTest, yPred)) )

Train error: $794.88
Test error: $799.26


In [27]:
features = ['cats_allowed', 'dogs_allowed', 'bedrooms', 'bathrooms', 
            'elevator', 'hardwood_floors', 'doorman', 'dishwasher', 'no_fee',
            'laundry_in_building', 'fitness_center', 'pre-war', 'laundry_in_unit',
            'roof_deck', 'outdoor_space', 'dining_room', 'high_speed_internet',
            'balcony', 'swimming_pool', 'new_construction', 'terrace', 
            'exclusive', 'loft', 'garden_patio', 'common_outdoor_space']

xTrain = train[ features]
xTest = test[ features]

linReg.fit( xTrain, yTrain)
yPred = linReg.predict( xTrain)
print( 'Train error: ${0:.2f}'.format( mean_absolute_error( yTrain, yPred)) )

yPred = linReg.predict( xTest)
print( 'Test error: ${0:.2f}'.format( mean_absolute_error( yTest, yPred)) )

Train error: $748.98
Test error: $757.54


In [31]:
print( 'Intercept\t\t', linReg.intercept_, '\n')
print( pd.Series( linReg.coef_, features).to_string() )

Intercept		 371.4271756215485 

cats_allowed             -95.522706
dogs_allowed             184.522455
bedrooms                 461.253558
bathrooms               1765.166078
elevator                 155.486893
hardwood_floors         -185.168163
doorman                  614.456396
dishwasher               114.173115
no_fee                  -187.634992
laundry_in_building     -257.969203
fitness_center           232.993854
pre-war                  -52.512753
laundry_in_unit          559.480281
roof_deck               -144.822962
outdoor_space           -120.931884
dining_room              159.490162
high_speed_internet     -307.345086
balcony                  -84.973709
swimming_pool             31.230072
new_construction        -115.852895
terrace                  225.388068
exclusive                 67.760835
loft                     209.802758
garden_patio             -58.702191
common_outdoor_space      45.492371


In [34]:
from sklearn.metrics import mean_squared_error, r2_score

yPred = linReg.predict( xTrain)
print( 'Root mean squared error of train data:', np.sqrt( mean_squared_error( yTrain, yPred)) )
print( 'Its mean absolute error:', mean_absolute_error( yTrain, yPred) )
print( 'And its R^2:', r2_score( yTrain, yPred) )

yPred = linReg.predict( xTest)
print( '\nRoot mean squared error of test data:', np.sqrt( mean_squared_error( yTest, yPred)) )
print( 'Its mean absolute error:', mean_absolute_error( yTest, yPred) )
print( 'And its R^2:', r2_score( yTest, yPred) )

Root mean squared error of train data: 1147.2093073364124
Its mean absolute error: 748.9802475676714
And its R^2: 0.5761425975918815

Root mean squared error of test data: 1136.500397037836
Its mean absolute error: 757.5358645783142
And its R^2: 0.5844194614023371
